This notebook seeks to explore the gender diversity of the different apache projects & the process

In [1]:
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import collect_set, explode, from_json
from pyspark.sql.types import *

import json
import os
import meetup.api
from copy import copy
import time
import logging

from watson_developer_cloud import ToneAnalyzerV3

API key configuration

In [2]:
meetup_key = os.getenv("MEETUP_APIKEY")
tone_bluemix_user = os.getenv("TONE_BLUEMIX_USERNAME")
tone_bluemix_password = os.getenv("TONE_BLUEMIX_PASSWORD")

Less secret configuration

In [3]:
max_meetup_events = 400

In [4]:
session = SparkSession.builder.appName("whatCanWeLearnFromTheSixties").getOrCreate()
sc = session.sparkContext

The first thing we want to get is the committers and PMC members, this information is stored in LDAP but also available in JSON. Eventually we will want to enrich this with mailing list information

In [5]:
def loadFlatJsonFile(path, explodeKey, schema=None):
    """Load a flat multi-line json file and convert into Spark & explode"""
    rdd = sc.wholeTextFiles(path).values()
    df = (session.read.schema(schema)
            .json(rdd))
    return df.select(explode(explodeKey))

In [25]:
apache_people_schema = StructType([StructField("lastCreateTimestamp", StringType()),
                     StructField("people",
                                 MapType(StringType(), 
                                         StructType([StructField('name', StringType()),
                                                     StructField('key_fingerprints', ArrayType(StringType())),
                                                     StructField('urls', ArrayType(StringType())),
                                                    ]))
                                )])
apache_people_df = loadFlatJsonFile(path="http_data_sources/public_ldap_people.json", # http://people.apache.org/public/public_ldap_people.json
                                 explodeKey="people", schema=apache_people_schema)
apache_people_df = apache_people_df.select(apache_people_df.key.alias("username"), apache_people_df.value.alias("extra"))

In [7]:
# Construct a lazy urllib3 pool
from lazy_helpers import *
    
bcast_pool = sc.broadcast(LazyPool)
bcast_pool.value

lazy_helpers.LazyPool

In [8]:
def project_on_github(project):
    """Returns if a project is on github"""
    import urllib3
    http = bcast_pool.value.get()
    r = http.request('GET', "https://github.com/apache/{0}".format(project))
    return r.status == 200
session.catalog.registerFunction("on_github", project_on_github, BooleanType())
# Except I'm a bad person so....
from pyspark.sql.catalog import UserDefinedFunction
project_on_github_udf = UserDefinedFunction(project_on_github, BooleanType(), "on_github")
session.catalog._jsparkSession.udf().registerPython("on_github", project_on_github_udf._judf)

In [9]:
apache_committees_schema = StructType([StructField("lastCreateTimestamp", StringType()),
                     StructField("committees",
                                 MapType(StringType(), StructType([StructField('roster', ArrayType(StringType())),
                                                                  StructField('modifyTimestamp', StringType()),
                                                                  StructField('createTimestamp', StringType())
                                                                  ])))])
apache_committees_df = loadFlatJsonFile(path="http_data_sources/public_ldap_committees.json", # http://people.apache.org/public/public_ldap_people.json
                                 explodeKey="committees", schema=apache_committees_schema)
apache_committees_on_github_df = apache_committees_df.filter(project_on_github_udf(apache_committees_df.key))
apache_committees_on_github_df.persist(StorageLevel.MEMORY_AND_DISK)
committee_names_df = apache_committees_on_github_df.select(apache_committees_df.key.alias("project"))
committee_names_df.persist(StorageLevel.MEMORY_AND_DISK)
committee_names_df.count()

148

In [26]:
project_to_user_df = apache_committees_on_github_df.select(
    apache_committees_on_github_df.key.alias("project"),
    explode(apache_committees_on_github_df.value.roster).alias("username"))


user_to_project_df = project_to_user_df.groupBy(project_to_user_df.username).agg(
    collect_set(project_to_user_df.project).alias("projects"))
apache_people_df = apache_people_df.join(user_to_project_df, on="username")

In [27]:
apache_people_df.take(1)

[Row(username='a_horuzhenko', extra=Row(name='Artyom Horuzhenko', key_fingerprints=None, urls=None), projects=['openmeetings'])]

Attempt to fetch relevant past & present meetups for each project - idea based on the listing at https://www.apache.org/events/meetups.html but different code

In [12]:
logger = logging.getLogger()
logger.setLevel("WARN")
def lookup_relevant_meetup(project_name, max_meetup_events=0):
    """Lookup relevant meetups for a specific project."""
    import logging
    import time
    import meetup.api
    logger = logging.getLogger()
    meetup_delay = 30
    meetup_reset_delay = 1800 # 30 minutes
    standard_keys = {"text_format": "plain", "trending": "desc=true", "and_text": "true", "city": "san francisco", "country": "usa", "text": "apache " + project_name, "radius": 10000}
    results = {"upcoming": [], "past": []}
    for status in ["upcoming", "past"]:
        keys = copy(standard_keys)
        keys["status"] = status
        count = 200
        base = 0
        while (count == 200 and (max_meetup_events == 0 or base < max_meetup_events)):
            logging.debug("Fetch {0} meetups for {1} on base {2}".format(status, project_name, base))
            project_name = "spark"
            client = client = meetup.api.Client(meetup_key)
            if base > 0:
                keys["page"] = base
            # Manually sleep for meetup_reset_delay on failure, the meetup-api package retry logic sometimes breaks :(
            response = None
            retry_count = 0
            while response is None and retry_count < 10:
                try:
                    response = client.GetOpenEvents(**keys)
                except:
                    response = None
                    retry_count += 1
                    time.sleep(meetup_reset_delay)
                    try:
                        response = client.GetOpenEvents(**keys)
                    except:
                        response = None
            try:
                count = response.meta['count']
                base = base + count
                results[status].append(response.results)
                time.sleep(meetup_delay)
            except:
                count = 0
    return (project_name, results)

project_meetups_df = committee_names_df.rdd.map(lambda x: x.project).map(lambda name: lookup_relevant_meetup(name, max_meetup_events))

In [ ]:
project_meetups_df.persist(StorageLevel.MEMORY_AND_DISK)
project_meetups_df.take(1)

For the provided projects attempt to lookup their GitHub

In [13]:
def lookup_project_git(project):
    """Returns the project github for a specific project. Assumes project is git hosted"""
    return "git://git.apache.org/{0}".format(project)
    

In [14]:
def fetch_project_github_data(project):
    from perceval.backends.core.github import GitHub as perceval_github
    gh_backend = perceval_github(owner="apache", repository=project)
    # The backend return a generator - which is awesome. However since we want to pull this data into Spark 
    return list(gh_backend.fetch())

In [15]:
def fetch_project_git_data(project):
    project_git = lookup_project_git(project)
    from perceval.backends.core.git import Git as perceval_git
    git_backend = perceval_git(owner="apache", repository=project)
    return list(gh_backend.fetch())

In [55]:
from lazy_helpers import *

bcast_driver = sc.broadcast(LazyDriver)

def lookup_crunchbase_info(people_and_projects):
    """Lookup a person a crunch base and see what the gender & company is.
    Filter for at least one mention of their projects."""
    from bs4 import BeautifulSoup
    import re
    driver = bcast_driver.value.get()
    for (username, name, projects) in people_and_projects:
        # robots.txt seems to be ok with person for now, double check before re-running this
        url = "https://www.crunchbase.com/person/{0}".format(name.replace(" ", "-"))
        try:
            driver.get(url)
            text = driver.page_source
            lower_text = text.lower()
            if any(project.lower() in lower_text for project in projects):
                soup = BeautifulSoup(text, "html.parser")
                stats = soup.findAll("div", { "class" : "info-card-overview-content"})[0]
                # Hacky but I'm lazy
                result = {}
                result["username"] = username
                try:
                    m = re.search("Gender:\</dt\>\<dd\>(.+?)\<", str(stats))
                    result["gender"] = m.group(1)
                except:
                    # If nothing matches thats ok
                    pass
                try:
                    m = re.search("data-name=\"(.+?)\" data-permalink=\"\/organization", str(stats))
                    result["company"] = m.group(1)
                except:
                    # No match no foul
                    pass
                yield result
        except:
            pass

In [56]:
result = lookup_crunchbase_info([("holden", "holden karau", ["spark"])])
list(result)

[{'company': 'IBM', 'gender': 'Female', 'username': 'holden'}]

Augment the committer info

In [58]:
# We do this as an RDD transformation since the cost of the transformation dominates
relevant_info = apache_people_df.select(
    apache_people_df.username,
    apache_people_df.extra.getField("name").alias("name"),
    apache_people_df.projects)
crunchbase_info_rdd = relevant_info.rdd.map(lambda row: (row.username, row.name, row.projects)).mapPartitions(lookup_crunchbase_info)
crunchbase_info_rdd.persist(StorageLevel.MEMORY_AND_DISK)
crunchbase_info_rdd.count()

18

In [61]:
crunchbase_info_rdd.collect()

[{'company': 'Hortonworks', 'gender': 'Male', 'username': 'ndimiduk'},
 {'company': 'Confluent', 'gender': 'Female', 'username': 'nehanarkhede'},
 {'company': 'Quantivate', 'gender': 'Male', 'username': 'nick'},
 {'company': 'Luminis Technologies', 'gender': 'Male', 'username': 'paulb'},
 {'company': 'Eventful', 'gender': 'Male', 'username': 'pramirez'},
 {'company': 'Yahoo', 'gender': 'Male', 'username': 'puru'},
 {'company': 'Databricks', 'gender': 'Male', 'username': 'pwendell'},
 {'company': 'WSO2', 'gender': 'Male', 'username': 'pzf'},
 {'company': 'Red Hat', 'gender': 'Male', 'username': 'rafabene'},
 {'company': 'Ohm Data', 'gender': 'Male', 'username': 'rawson'},
 {'company': 'Hortonworks', 'gender': 'Male', 'username': 'rbalamohan'},
 {'company': 'Apache Software Foundation',
  'gender': 'Male',
  'username': 'rbowen'},
 {'company': 'Schuberg Philis', 'gender': 'Male', 'username': 'remi'},
 {'company': 'Data Artisans', 'gender': 'Male', 'username': 'rmetzger'},
 {'company': 'S

One of the things that is interesting is understanding what the tones of the meetup descriptions & mailing list posts are. We can use https://www.ibm.com/watson/developercloud/tone-analyzer/api/v3/?python#introduction

In [ ]:
def lookup_tone(document):
    """Looks up the tone for a specific document. Returns a json blob."""
    from watson_developer_cloud import ToneAnalyzerV3
    tone_analyzer = ToneAnalyzerV3(
        username=tone_bluemix_user,
        password=tone_bluemix_password,
        version='2016-05-19 ')
    return tone_analyzer.tone(text=document)

In [ ]:
oh_no_you = lookup_tone("oh no you didn't girl")

In [ ]:
oh_no_you

Ok its time to find some mailing list info

In [62]:
def fetch_mbox_data(project_name):
    import tempfile
    import shutil
    from perceval.backends.core.mbox import MBox as perceval_mbox
    mbox_user_url = "http://mail-archives.apache.org/mod_mbox/{0}-user".format(project_name)
    mbox_dev_url = "http://mail-archives.apache.org/mod_mbox/{0}-dev".format(project_name)
    def fetch_mbox_from_apache_site(root_url):
        """Fetches all of the mbox files from a given apache url"""
        # Make a temp directory to hold the mbox files, we will return this directory
        tempdir = tempfile.mkdtemp()
        
        # Fetch the page to parse
        pool = bcast_pool.value.get()
        result = pool.request('GET', root_url)
        
        
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(result.data, "html.parser")
        mbox_ids = set(map(lambda tag: tag.get('id'), soup.findAll("span", { "class" : "links"})))
        
        for mbox_id in mbox_ids:
            mbox_url = "{0}/{1}.mbox".format(root_url, mbox_id)
            filename = "{0}/{1}.mbox".format(tempdir, mbox_id)
            print("fetching {0}".format(mbox_url))
            with pool.request('GET', mbox_url, preload_content=False) as r, open(filename, 'wb') as out_file:       
                shutil.copyfileobj(r, out_file)
        
        return tempdir
    def process_mbox_directory(base_url, dir_path):
        mbox_backend = perceval_mbox(base_url, dir_path)
        return mbox_backend.fetch()

    def load_and_process(base_url):
        output_dir = fetch_mbox_from_apache_site(base_url)
        return list(process_mbox_directory(base_url, output_dir))
    
    return list(map(load_and_process, [mbox_user_url, mbox_dev_url]))

In [ ]:
fetched_mbox_data = fetch_mbox_data("spark")

In [64]:
mailing_list_posts = committee_names_df.rdd.map(lambda row: fetch_mbox_data(row.project))
mailing_list_posts.persist(StorageLevel.MEMORY_AND_DISK)

PythonRDD[260] at RDD at PythonRDD.scala:48

In [65]:
mailing_list_posts.count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 58.0 failed 1 times, most recent failure: Lost task 0.0 in stage 58.0 (TID 3839, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:168)
	at org.apache.spark.api.python.PythonRunner$$anon$1.next(PythonRDD.scala:156)
	at org.apache.spark.api.python.PythonRunner$$anon$1.next(PythonRDD.scala:152)
	at org.apache.spark.InterruptibleIterator.next(InterruptibleIterator.scala:43)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsBytes(MemoryStore.scala:372)
	at org.apache.spark.storage.BlockManager$$anonfun$doPutIterator$1.apply(BlockManager.scala:1003)
	at org.apache.spark.storage.BlockManager$$anonfun$doPutIterator$1.apply(BlockManager.scala:978)
	at org.apache.spark.storage.BlockManager.doPut(BlockMa

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 48307)
----------------------------------------


Traceback (most recent call last):
  File "/usr/lib/python3.4/socketserver.py", line 305, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.4/socketserver.py", line 331, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.4/socketserver.py", line 344, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.4/socketserver.py", line 673, in __init__
    self.handle()
  File "/usr/local/lib/python3.4/dist-packages/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/usr/local/lib/python3.4/dist-packages/pyspark/serializers.py", line 594, in read_int
    raise EOFError
EOFError


In [66]:
apache_people_df.count()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.4/dist-packages/py4j/java_gateway.py", line 1035, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.4/dist-packages/py4j/java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.4/dist-packages/py4j/java_gateway.py", line 1040, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57929)
Traceback (most recent call last):
  File "/usr/local/lib/python3.4/dist-packages/py4j/java_gateway.py", line 827, in _get_connection
    con

Py4JError: An error occurred while calling o494.count

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57929)
Traceback (most recent call last):
  File "/usr/local/lib/python3.4/dist-packages/py4j/java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.4/dist-packages/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57929)
Traceback (most recent call last):
  File "/usr/local/lib/python3.4/dist-packages/py4j/java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most 